# PyCUDA与图像处理之HelloWorld

In [7]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy as np
from pycuda.compiler import SourceModule
import string

In [20]:
def load_kernel_from_file(fname,kname):
    sources = None
    kernel = None
    with open(fname,encoding='utf-8') as f:
        sources = str(f.read())
        sm = SourceModule(sources)
        kernel = sm.get_function(kname)
    return kernel

def load_kernel_from_string(sources,kname):
    sm = SourceModule(str(sources))
    kernel = sm.get_function(kname)
    return kernel

In [18]:
sources = string.Template("""
    __global__ void multiply_array(float * dest,float * a,float *b){
        const int i = threadIdx.x;
        dest[i] = a[i] * b[i];
        }
""").substitute()

In [21]:
multiply_array = load_kernel_from_string(sources,'multiply_array')

a = np.array([1,2,3,4,5,6,7,8,9]).astype(np.float32)
b = np.array([1,2,3,4,5,6,7,8,9]).astype(np.float32)

dest = np.zeros_like(a)

multiply_array(drv.Out(dest),drv.In(a),drv.In(b),block = (9,1,1),grid=(1,1))
print(dest)

[ 1.  4.  9. 16. 25. 36. 49. 64. 81.]
